In [82]:
import pandas as pd
import xlrd


In [261]:
# Read CH data from excel
ch = pd.read_excel('../data/Cadre Harmonise/cadre_harmonise.xlsx')


In [262]:
# Features I'm interested in:
ch = ch[['adm0_name','adm1_name','adm2_name','population','phase_class','phase35',
        'chtype','exercise_label','exercise_year','reference_label','reference_year']]

# Select adm0_name to work with
ch = ch[ch.adm0_name.isin(['Mali','Burkina Faso','Niger'])]
# Select adm1_name to work with
ch = ch[ch.adm1_name.isin(['Gao','Mopti','Tombouctou','Nord','Sahel','Est','Tahoua','Tillaberi'])]


ch = ch.sort_values(by=['reference_year','adm0_name','adm1_name','adm2_name','reference_label','exercise_year','exercise_label','chtype'])
ch = ch[ch.reference_label.isin(['Jun-Aug'])]
ch = ch[ch.reference_year == ch.exercise_year]
ch = ch.reset_index(drop=True)


for i in ch.index:
    if ch.loc[i,'adm2_name']=='Tahoua_accessible':
        ch.at[i,'adm2_name']='Tahoua'
    if ch.loc[i,'adm2_name']=='Tillaberi_accessible':
        ch.at[i,'adm2_name']='Tillaberi'
    if ch.loc[i,'adm2_name']=='Tera_accessible':
        ch.at[i,'adm2_name']='Tera'
    if ch.loc[i,'adm2_name']=='Ouallam_accessible':
        ch.at[i,'adm2_name']='Ouallam'
        
        
        
        
# Filingue = Filingue + Abala + Balleyara
filingue = ch.loc[ch.adm2_name=='Filingue']
abala = ch.loc[ch.adm2_name=='Abala']
balleyara = ch.loc[ch.adm2_name=='Balleyara']
# NOTE: It MUST have the same index
abala.index += 5
balleyara.index += 3
for year in range(2014,2021):
    ch.loc[(ch.adm2_name=='Filingue')&(ch.reference_year==year),'phase_class']=pd.concat([filingue.loc[filingue.reference_year==year, 'phase_class'], abala.loc[abala.reference_year==year, 'phase_class'], balleyara.loc[balleyara.reference_year==year, 'phase_class']]).max(level=0)
    ch.loc[(ch.adm2_name=='Filingue')&(ch.reference_year==year),'population']+=abala.loc[abala.reference_year==year, 'population'] + balleyara.loc[balleyara.reference_year==year, 'population']
    ch.loc[(ch.adm2_name=='Filingue')&(ch.reference_year==year),'phase35']+=abala.loc[abala.reference_year==year, 'phase35'] + balleyara.loc[balleyara.reference_year==year, 'phase35']
    

# Tera = Tera + Tera_limitedaccess + Bankilare
tera = ch.loc[ch.adm2_name=='Tera']
tera_la = ch.loc[ch.adm2_name=='Tera_limitedaccess']
bankilare = ch.loc[ch.adm2_name=='Bankilare']
# NOTE: It MUST have the same index
tera_la.index -= 1
bankilare.index += 6
for year in range(2014,2021):
    if year!=2020:
        ch.loc[(ch.adm2_name=='Tera')&(ch.reference_year==year),'phase_class']=pd.concat([tera.loc[tera.reference_year==year, 'phase_class'], bankilare.loc[bankilare.reference_year==year, 'phase_class']]).max(level=0)
    else:
        bankilare.index += 1
    ch.loc[(ch.adm2_name=='Tera')&(ch.reference_year==year),'population']+=bankilare.loc[bankilare.reference_year==year, 'population'] 
    ch.loc[(ch.adm2_name=='Tera')&(ch.reference_year==year),'phase35']+=bankilare.loc[bankilare.reference_year==year, 'phase35']
ch.loc[(ch.adm2_name=='Tera')&(ch.reference_year==2020),'phase_class']=pd.concat([tera.loc[tera.reference_year==2020, 'phase_class'], tera_la.loc[tera_la.reference_year==2020, 'phase_class'], bankilare.loc[bankilare.reference_year==2020, 'phase_class']]).max(level=0)
ch.loc[(ch.adm2_name=='Tera')&(ch.reference_year==2020),'population']+=tera_la.loc[tera_la.reference_year==2020, 'population'].values[0]
ch.loc[(ch.adm2_name=='Tera')&(ch.reference_year==2020),'phase35']+=tera_la.loc[tera_la.reference_year==2020, 'phase35'].values[0]


# Say = Say + Torodi
say = ch.loc[ch.adm2_name=='Say']
torodi = ch.loc[ch.adm2_name=='Torodi']
# NOTE: It MUST have the same index
torodi.index -= 3
for year in range(2014,2021):
    if year==2018:
        torodi.index-=1
    if year==2020:
        torodi.index-=2
    ch.loc[(ch.adm2_name=='Say')&(ch.reference_year==year),'phase_class']=pd.concat([say.loc[say.reference_year==year, 'phase_class'], torodi.loc[torodi.reference_year==year, 'phase_class']]).max(level=0)
    ch.loc[(ch.adm2_name=='Say')&(ch.reference_year==year),'population']+=torodi.loc[torodi.reference_year==year, 'population'] 
    ch.loc[(ch.adm2_name=='Say')&(ch.reference_year==year),'phase35']+=torodi.loc[torodi.reference_year==year, 'phase35'] 
    if year==2018:
        torodi.index+=1


# Ouallam = Ouallam + Ouallam_limitedaccess
ouallam = ch.loc[ch.adm2_name=='Ouallam']
ouallam_la = ch.loc[ch.adm2_name=='Ouallam_limitedaccess']
# NOTE: It MUST have the same index
ouallam_la.index -= 1
ch.loc[(ch.adm2_name=='Ouallam')&(ch.reference_year==2020),'phase_class']=pd.concat([ouallam.loc[ouallam.reference_year==2020, 'phase_class'], ouallam_la.loc[ouallam_la.reference_year==2020, 'phase_class']]).max(level=0)
ch.loc[(ch.adm2_name=='Ouallam')&(ch.reference_year==2020),'population']+=ouallam_la.loc[ouallam_la.reference_year==2020, 'population'].values[0]
ch.loc[(ch.adm2_name=='Ouallam')&(ch.reference_year==2020),'phase35']+=ouallam_la.loc[ouallam_la.reference_year==2020, 'phase35'].values[0]

    
# Tillaberi = Tillaberi + Tillaberi_limitedaccess + Tillaberi Commune + Ayerou + Gotheye
tillaberi = ch.loc[ch.adm2_name=='Tillaberi']
tillaberi_la = ch.loc[ch.adm2_name=='Tillaberi_limitedaccess']
tillaberi_commune = ch.loc[ch.adm2_name=='Tillaberi Commune']
ayerou = ch.loc[ch.adm2_name=='Ayerou']
gotheye = ch.loc[ch.adm2_name=='Gotheye']
# NOTE: It MUST have the same index
tillaberi_la.index -= 1
tillaberi_commune.index -= 1
ayerou.index += 10
gotheye.index += 5
for year in range(2014,2021):
    if year!=(2018&2020):
        ch.loc[(ch.adm2_name=='Tillaberi')&(ch.reference_year==year),'phase_class']=pd.concat([tillaberi.loc[tillaberi.reference_year==year, 'phase_class'], ayerou.loc[ayerou.reference_year==year, 'phase_class'], gotheye.loc[gotheye.reference_year==year, 'phase_class']]).max(level=0)
    if year==2020:
        ayerou.index += 2
        gotheye.index += 2
    ch.loc[(ch.adm2_name=='Tillaberi')&(ch.reference_year==year),'population']+=ayerou.loc[ayerou.reference_year==year, 'population'] + gotheye.loc[gotheye.reference_year==year, 'population'] 
    ch.loc[(ch.adm2_name=='Tillaberi')&(ch.reference_year==year),'phase35']+=ayerou.loc[ayerou.reference_year==year, 'phase35'] + gotheye.loc[gotheye.reference_year==year, 'phase35']
ch.loc[(ch.adm2_name=='Tillaberi')&(ch.reference_year==2018),'phase_class']=pd.concat([tillaberi.loc[tillaberi.reference_year==2018, 'phase_class'], tillaberi_commune.loc[tillaberi_commune.reference_year==2018, 'phase_class'], ayerou.loc[ayerou.reference_year==year, 'phase_class'], gotheye.loc[gotheye.reference_year==year, 'phase_class']]).max(level=0)
ch.loc[(ch.adm2_name=='Tillaberi')&(ch.reference_year==2018),'population']+=tillaberi_commune.loc[tillaberi_commune.reference_year==2018, 'population'].values[0]
ch.loc[(ch.adm2_name=='Tillaberi')&(ch.reference_year==2018),'phase35']+=tillaberi_commune.loc[tillaberi_commune.reference_year==2018, 'phase35'].values[0]
ch.loc[(ch.adm2_name=='Tillaberi')&(ch.reference_year==2020),'phase_class']=pd.concat([tillaberi.loc[tillaberi.reference_year==2020, 'phase_class'], tillaberi_la.loc[tillaberi_la.reference_year==2020, 'phase_class'], ayerou.loc[ayerou.reference_year==year, 'phase_class'], gotheye.loc[gotheye.reference_year==year, 'phase_class']]).max(level=0)
ch.loc[(ch.adm2_name=='Tillaberi')&(ch.reference_year==2020),'population']+=tillaberi_la.loc[tillaberi_la.reference_year==2020, 'population'].values[0]
ch.loc[(ch.adm2_name=='Tillaberi')&(ch.reference_year==2020),'phase35']+=tillaberi_la.loc[tillaberi_la.reference_year==2020, 'phase35'].values[0]


# Tahoua = Tahoua + Tahoua_limitedaccess + Ville De Tahoua
tahoua = ch.loc[ch.adm2_name=='Tahoua']
ville_tahoua = ch.loc[ch.adm2_name=='Ville De Tahoua']
tahoua_la = ch.loc[ch.adm2_name=='Tahoua_limitedaccess']
# NOTE: It MUST have the same index
tahoua_la.index -= 1
ville_tahoua.index -= 4
ch.loc[(ch.adm2_name=='Tahoua')&(ch.reference_year==2018),'phase_class']=pd.concat([tahoua.loc[tahoua.reference_year==2018, 'phase_class'], ville_tahoua.loc[ville_tahoua.reference_year==2018, 'phase_class']]).max(level=0)
ch.loc[(ch.adm2_name=='Tahoua')&(ch.reference_year==2018),'population']+=ville_tahoua.loc[ville_tahoua.reference_year==2018, 'population'].values[0]
ch.loc[(ch.adm2_name=='Tahoua')&(ch.reference_year==2018),'phase35']+=ville_tahoua.loc[ville_tahoua.reference_year==2018, 'phase35'].values[0]
ch.loc[(ch.adm2_name=='Tahoua')&(ch.reference_year==2020),'phase_class']=pd.concat([tahoua.loc[tahoua.reference_year==2020, 'phase_class'], tahoua_la.loc[tahoua_la.reference_year==2020, 'phase_class']]).max(level=0)
ch.loc[(ch.adm2_name=='Tahoua')&(ch.reference_year==2020),'population']+=tahoua_la.loc[tahoua_la.reference_year==2020, 'population'].values[0]
ch.loc[(ch.adm2_name=='Tahoua')&(ch.reference_year==2020),'phase35']+=tahoua_la.loc[tahoua_la.reference_year==2020, 'phase35'].values[0]


# Tassara = Tassara + Tchintabaraden 
tassara = ch.loc[ch.adm2_name=='Tassara']
tchintabaraden = ch.loc[ch.adm2_name=='Tchintabaraden']
# NOTE: It MUST have the same index
tchintabaraden.index -= 1
for year in range(2014,2021):
    ch.loc[(ch.adm2_name=='Tassara')&(ch.reference_year==year),'phase_class']=pd.concat([tassara.loc[tassara.reference_year==year, 'phase_class'], tchintabaraden.loc[tchintabaraden.reference_year==year, 'phase_class']]).max(level=0)
    ch.loc[(ch.adm2_name=='Tassara')&(ch.reference_year==year),'population']+=tchintabaraden.loc[tchintabaraden.reference_year==year, 'population'] 
    ch.loc[(ch.adm2_name=='Tassara')&(ch.reference_year==year),'phase35']+=tchintabaraden.loc[tchintabaraden.reference_year==year, 'phase35'] 

        
       
    
# Select adm2_name to work with
ch = ch[ch.adm2_name.isin(['Bandiagara','Bankass','Djenne','Douentza','Koro','Mopti','Tenenkou','Youwarou',
                           'Dire','Goundam','Gourma-Rharous','Niafunke','Tombouctou',
                           'Ansongo','Bourem','Gao','Menaka',
                           'Komonjdjari',
                           'Loroum','Yatenga',
                           'Oudalan','Seno','Soum','Yagha',
                           'Tahoua','Tassara','Tillia', # I put in Tillaberi and Tahoua considering that some values are in Tillaberi Department and Tahoua Department, Ville De Tahoua, Tillaberi Commune
                           'Banibangou','Filingue','Kollo','Ouallam','Say','Tera','Tillaberi'])]#removed: Torodi, Ayerou, Abala, Gotheye, Bankilare, Balleyara Because considered in the other adm2

   
ch = ch.reset_index(drop=True)


# Check that no row has a nan value:
ch[ch.isna().any(axis=1)]

ch
        
        
        



,adm0_name,adm1_name,adm2_name,population,phase_class,phase35,chtype,exercise_label,exercise_year,reference_label,reference_year
0,Burkina Faso,Est,Komonjdjari,105540.0,3.0,21108.0,projected,Jan-May,2014,Jun-Aug,2014
1,Burkina Faso,Nord,Loroum,177109.0,2.0,0.0,projected,Jan-May,2014,Jun-Aug,2014
2,Burkina Faso,Nord,Yatenga,678703.0,2.0,0.0,projected,Jan-May,2014,Jun-Aug,2014
3,Burkina Faso,Sahel,Oudalan,251998.0,3.0,50400.0,projected,Jan-May,2014,Jun-Aug,2014
4,Burkina Faso,Sahel,Seno,331150.0,3.0,66230.0,projected,Jan-May,2014,Jun-Aug,2014
5,Burkina Faso,Sahel,Soum,440647.0,3.0,88129.0,projected,Jan-May,2014,Jun-Aug,2014
6,Burkina Faso,Sahel,Yagha,203315.0,2.0,0.0,projected,Jan-May,2014,Jun-Aug,2014
7,Mali,Gao,Ansongo,157184.0,3.0,56586.0,projected,Jan-May,2014,Jun-Aug,2014
8,Mali,Gao,Bourem,138610.0,4.0,62374.0,projected,Jan-May,2014,Jun-Aug,2014
9,Mali,Gao,Gao,285337.0,3.0,156935.0,projected,Jan-May,2014,Jun-Aug,2014


,adm0_name,adm1_name,adm2_name,population,phase_class,phase35,chtype,exercise_label,exercise_year,reference_label,reference_year


In [258]:
#pd.DataFrame({'phase35' : ch.groupby(['adm2_name','reference_year'])['phase35'].sum()})
ch.loc[ch.adm2_name=='Say']

,adm0_name,adm1_name,adm2_name,population,phase_class,phase35,chtype,exercise_label,exercise_year,reference_label,reference_year
51,Niger,Tillaberi,Say,389966.0,2.0,20299.0,projected,Jan-May,2014,Jun-Aug,2014
106,Niger,Tillaberi,Say,393594.0,1.0,4013.0,projected,Jan-May,2015,Jun-Aug,2015
161,Niger,Tillaberi,Say,412827.0,2.0,8418.0,projected,Jan-May,2016,Jun-Aug,2016
216,Niger,Tillaberi,Say,428910.0,2.0,15137.0,projected,Jan-May,2017,Jun-Aug,2017
272,Niger,Tillaberi,Say,445826.0,1.0,0.0,projected,Jan-May,2018,Jun-Aug,2018
328,Niger,Tillaberi,Say,463383.0,3.0,66049.0,projected,Jan-May,2019,Jun-Aug,2019
398,Niger,Tillaberi,Say,481601.0,3.0,70631.0,projected,Jan-May,2020,Jun-Aug,2020


In [255]:
ch.loc[ch.adm2_name=='Tera_limitedaccess']

,adm0_name,adm1_name,adm2_name,population,phase_class,phase35,chtype,exercise_label,exercise_year,reference_label,reference_year
400,Niger,Tillaberi,Tera_limitedaccess,89099.0,3.0,20493.0,projected,Jan-May,2020,Jun-Aug,2020


,adm0_name,adm1_name,adm2_name,population,phase_class,phase35,chtype,exercise_label,exercise_year,reference_label,reference_year
53,Niger,Tillaberi,Tillaberi,560655.0,2.0,15078.0,projected,Jan-May,2014,Jun-Aug,2014
108,Niger,Tillaberi,Tillaberi,577261.0,2.0,19816.0,projected,Jan-May,2015,Jun-Aug,2015
163,Niger,Tillaberi,Tillaberi,539209.0,1.0,14219.0,projected,Jan-May,2016,Jun-Aug,2016
218,Niger,Tillaberi,Tillaberi,560212.0,2.0,20205.0,projected,Jan-May,2017,Jun-Aug,2017
274,Niger,Tillaberi,Tillaberi,653890.0,2.0,26189.0,projected,Jan-May,2018,Jun-Aug,2018
330,Niger,Tillaberi,Tillaberi,617969.0,3.0,74058.0,projected,Jan-May,2019,Jun-Aug,2019
401,Niger,Tillaberi,Tillaberi,NaN,3.0,NaN,projected,Jan-May,2020,Jun-Aug,2020


In [221]:
ch.loc[ch.adm2_name=='Tillaberi']


,adm0_name,adm1_name,adm2_name,population,phase_class,phase35,chtype,exercise_label,exercise_year,reference_label,reference_year
53,Niger,Tillaberi,Tillaberi,560655.0,2.0,15078.0,projected,Jan-May,2014,Jun-Aug,2014
108,Niger,Tillaberi,Tillaberi,577261.0,2.0,19816.0,projected,Jan-May,2015,Jun-Aug,2015
163,Niger,Tillaberi,Tillaberi,539209.0,1.0,14219.0,projected,Jan-May,2016,Jun-Aug,2016
218,Niger,Tillaberi,Tillaberi,560212.0,2.0,20205.0,projected,Jan-May,2017,Jun-Aug,2017
274,Niger,Tillaberi,Tillaberi,653890.0,2.0,26189.0,projected,Jan-May,2018,Jun-Aug,2018
330,Niger,Tillaberi,Tillaberi,617969.0,3.0,74058.0,projected,Jan-May,2019,Jun-Aug,2019
401,Niger,Tillaberi,Tillaberi,642265.0,3.0,73625.0,projected,Jan-May,2020,Jun-Aug,2020


In [263]:
tillaberi_la.loc[tillaberi_la.reference_year==2020, 'population'].values[0]



34097.0

In [256]:
ch.loc[ch.adm2_name=='Bankilare']


,adm0_name,adm1_name,adm2_name,population,phase_class,phase35,chtype,exercise_label,exercise_year,reference_label,reference_year
46,Niger,Tillaberi,Bankilare,90740.0,2.0,9074.0,projected,Jan-May,2014,Jun-Aug,2014
101,Niger,Tillaberi,Bankilare,93268.0,2.0,3731.0,projected,Jan-May,2015,Jun-Aug,2015
156,Niger,Tillaberi,Bankilare,97829.0,1.0,8805.0,projected,Jan-May,2016,Jun-Aug,2016
211,Niger,Tillaberi,Bankilare,101640.0,2.0,4066.0,projected,Jan-May,2017,Jun-Aug,2017
267,Niger,Tillaberi,Bankilare,105649.0,3.0,24299.0,projected,Jan-May,2018,Jun-Aug,2018
323,Niger,Tillaberi,Bankilare,109810.0,2.0,7687.0,projected,Jan-May,2019,Jun-Aug,2019
392,Niger,Tillaberi,Bankilare,114127.0,3.0,28532.0,projected,Jan-May,2020,Jun-Aug,2020


In [5]:
#Quello che segue in "" è sbagliato
"""
#filingue = ch[ch.adm2_name.isin(['Filingue','Abala','Balleyara'])].sort_values(by=['reference_year','adm0_name','adm1_name','reference_label','exercise_year','chtype','adm2_name'])
#ch[ch.adm2_name.isin(['Say','Torodi'])].sort_values(by=['reference_year','adm0_name','adm1_name','reference_label','exercise_year','chtype','adm2_name'])
#ch[ch.adm2_name.isin(['Tera','Bankilare'])].sort_values(by=['reference_year','adm0_name','adm1_name','reference_label','exercise_year','chtype','adm2_name'])
#ch[ch.adm2_name.isin(['Tillaberi','Ayerou','Gotheye'])].sort_values(by=['reference_year','adm0_name','adm1_name','reference_label','exercise_year','chtype','adm2_name'])
#ch[ch.adm2_name.isin(['Filingue','Abala','Balleyara'])].sort_values(by=['reference_year','adm0_name','adm1_name','reference_label','exercise_year','chtype','adm2_name'])
"""

#pd.DataFrame({'total_fatalities' : my_conf.groupby(['reference_year','adm0_name','adm1_name','adm2_name','actor1'])['fatalities'].sum(),
#              'mean_fatalities' : my_conf.groupby(['reference_year','adm0_name','adm1_name','adm2_name','actor1'])['fatalities'].mean(), 
#              'attack_numb' : my_conf.groupby(['reference_year','adm0_name','adm1_name','adm2_name','actor1'])['fatalities'].count()}).sort_values(by=['total_fatalities','attack_numb','mean_fatalities'],ascending=False)
pop = pd.DataFrame({'population' : ch.groupby(['reference_year','reference_label','exercise_year','exercise_label','chtype'])['population'].sum()})
pop
#ch[ch.adm2_name.isin(['Filingue'])]['population'] 
#ch[ch.adm2_name.isin(['Filingue'])]
# Sorting ch by ['adm0_name','reference_year','chtype','reference_label','exercise_label','adm1_name','adm2_name']
#ch = ch.sort_values(by=['reference_year','adm0_name','adm1_name','adm2_name','reference_label','exercise_year','chtype'])
#ch = ch.reset_index(drop=True)


#ch

# Look for all different adm2_name
#pd.DataFrame({'ad' :ch.groupby(['adm0_name','adm1_name','adm2_name'])['phase35'].mean()})

population
reference_year reference_label exercise_year exercise_label chtype               
2014           Jan-May         2014          Jan-May        current      14540342
               Jun-Aug         2014          Jan-May        projected    14540342
               Sep-Dec         2014          Sep-Dec        current      14659110
2015           Jan-May         2014          Sep-Dec        projected    14843110
                               2015          Jan-May        current      14841775
               Jun-Aug         2015          Jan-May        projected    14841775
               Sep-Dec         2015          Sep-Dec        current      15292867
2016           Jan-May         2016          Jan-May        current      15381081
               Jun-Aug         2015          Sep-Dec        projected    15292867
                               2016          Jan-May        projected    15381081
               Sep-Dec         2016          Sep-Dec        current      15718766
2017           Jan-May         2017          Jan-May        current      15837665
               Jun-Aug         2016          Sep-Dec        projected    15718766
                               2017          Jan-May        projected    15837665
               Sep-Dec         2017          Sep-Dec        current      16501958
2018           Jan-May         2018          Jan-May        current      16501958
               Jun-Aug         2017          Sep-Dec        projected    16501958
                               2018          Jan-May        projected    16501958
               Sep-Dec         2018          Sep-Dec        current      16799472
2019           Jan-May         2019          Jan-May        current      16799472
               Jun-Aug         2018          Sep-Dec        projected    16799472
                               2019          Jan-May        projected    16799472

In [ ]:
"""
# Features I'm interested in:
ch = ch[['adm0_name','adm1_name','adm2_name','population','phase_class','phase35',
        'chtype','exercise_label','exercise_year','reference_label','reference_year']]

# Replace names to make consistent data: Tahoua, Tillaberi. THIS IS NOT WORKING, I take out Tillaberi and Tahoua to postpone the problem
#ch['adm2_name'].replace('Tahoua Department','Tahoua',inplace=True)
#ch['adm2_name'].replace('Tillaberi Department','Tillaberi',inplace=True)

# Select adm0_name to work with
ch = ch[ch.adm0_name.isin(['Mali','Burkina Faso','Niger'])]

# Select adm1_name to work with
ch = ch[ch.adm1_name.isin(['Gao','Mopti','Tombouctou','Nord','Sahel','Est','Tahoua','Tillaberi'])]

# Select adm2_name to work with
ch = ch[ch.adm2_name.isin(['Bankass','Koro','Douentza','Djenne','Bandiagara','Tenenkou','Mopti','Youwarou',
                              'Gourma-Rharous','Dire','Niafunke',
                              'Gao','Ansongo','Menaka','Bourem',
                              'Yatenga','Loroum',
                              'Yagha','Seno','Soum','Oudalan',
                              'Komonjdjari',
                              'Tillia', # I take out Tillaberi and Tahoua because of some values are in Tillaberi Department and Tahoua Department
                              'Banibangou','Filingue','Ouallam','Say','Tera','Torodi','Abala','Ayerou'])]#removed: Tassara, Gotheye, Bankilare, Balleyara Because are not in conflict

# Sorting ch by ['adm0_name','reference_year','chtype','reference_label','exercise_label','adm1_name','adm2_name']
ch = ch.sort_values(by=['adm0_name','reference_year','chtype','reference_label','exercise_label','adm1_name','adm2_name'])
ch = ch.reset_index(drop=True)
"""

In [ ]:
"""
# We want to work just with the Lean Season data
# We need just the spring estimation: Select exercise_label=='Jan-May'
lean = ch[ch.exercise_label == 'Jan-May']
# Select reference_label='Jun-Aug'
lean = lean[lean.reference_label == 'Jun-Aug']
# Select chtype=='projected'
lean = lean[lean.chtype == 'projected']
lean = lean.reset_index(drop=True)
# Select featueres
lean = lean[['adm0_name','adm1_name','adm2_name','population','phase_class','phase35','reference_year']]

# Let's plot things out
# phase35 density between the population in Lean Season: (phase35%, reference_year)
lean['p35_density'] = lean['phase35'].div(lean['population'])
#lean_graph = lean[lean.adm2_name=='Gao'].plot(x='reference_year',y='p35_density',figsize=(12,8))
"""